#/home/user/data/container/train+test+val 10K aug+ 1636 real

In [1]:
import keras
import tensorflow as tf
import sys
import os
print('TensorFlow version:', tf.__version__)
print('Keras version:', keras.__version__)

import cv2

Using TensorFlow backend.


TensorFlow version: 1.12.0
Keras version: 2.2.4


In [2]:
DEVICE = "/gpu:0"  

working_dir = os.getcwd()
print("Working directory is:",working_dir)
sys.path.append(working_dir)  # To find local version of the library

linux= False

if linux:
    ROOT_DIR = os.path.join("/home/user/gullseye/sentinel") #linux        
    DATA_DIR = "/home/user/data/evrim" 
else:    
    ROOT_DIR = os.path.join("C:/Users/is95217/PycharmProjects/source/turkishplate/_1 Container/") #win
    DATA_DIR = "C:/Users/is95217/data" #win
    
sys.path.append(ROOT_DIR + "/api")

import Modelopcontainer as p # 

print("Data:", DATA_DIR)
print("Source code:", ROOT_DIR)

Working directory is: C:\Users\is95217\PycharmProjects\source\turkishplate\_1 Container
Data: C:/Users/is95217/data
Source code: C:/Users/is95217/PycharmProjects/source/turkishplate/_1 Container/


In [ ]:
DEVICE

# Model description and training

In [3]:
lrate=0.02
ibatchsize=32

In [4]:

#model2 = p.train2gru(128,32, #imgw,imgh  #HEİGHT küçültüldü
#model2 = p.train2gru(128,32,
model2 = p.train2gru(128,64, #imgw,imgh  --> SONRA BUNU YAP                     
                     DATA_DIR, #path
                     False,         #load             
                     ROOT_DIR + "/weights/model_container_out",  #path & name_model
                     1, #epoch
                     lrate,#learning rate
                     ibatchsize) 

#out-> Epoch 30/30 8582/8582 loss: 0.0027 - acc: 0.9990 - val_loss: 0.7792 - val_acc: 0.9124

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 64, 16)  0           conv1[0][0]                      
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           dropout_1[0][0]                  
__________________________________________________________________________________________________
conv2 (Con

In [ ]:
import matplotlib.pyplot as plt
# Get training and test loss histories
training_loss = model2.history.history['loss']
test_loss = model2.history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();


In [ ]:
training_acc = model2.history.history['acc']
test_acc = model2.history.history['val_acc']
# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training acc', 'Test acc'])
plt.xlabel('Epoch')
plt.ylabel('acc')
plt.show();

# Load MODEL

In [ ]:
import concurrent.futures
import numpy as np
import cv2
import keras.backend as K
from keras.layers import Dense
from keras.models import Sequential

import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.backend.tensorflow_backend import set_session

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
tf_config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)

In [ ]:
graph_detection = tf.Graph()

with graph_detection.as_default():
    session_detection = tf.Session(graph = graph_detection, config=tf_config) 
    with session_detection.as_default():
        #load model
        loaded_model=p.load_model(ROOT_DIR +'/weights/model_container_out',lrate) #mix pics from port and augmentation

# Test on validation images

In [ ]:
DATA_DIR

In [ ]:
#img_filepath = DATA_DIR + '/test/ACJG6440259.jpg' #ok
#img_filepath = DATA_DIR + '/test/APZU3711995.R 20190315233027.jpg' #not ok 
img_filepath = DATA_DIR + '/test/ADMU12061392.jpg' #not ok -> DRHU6747850
image = cv2.imread(img_filepath)

with graph_detection.as_default():
     with session_detection.as_default():
            #predicted_text = p.predict_single_plate(session_detection,loaded_model,image,128,32)#input directory of pic
            predicted_text = p.predict_single_plate(session_detection,loaded_model,image,128,64)#input directory of pic
            print(predicted_text)

# Full test set predict

In [ ]:
img_filepath=DATA_DIR + '/test/' #sonda slash olacak!
#img_filepath = '/home/user/data/container/4straightgullseye -ters/' 
#img_filepath= '/home/user/data/container/4straightgullseye/'

cnt_test= len(os.listdir(img_filepath))
cnt_test

In [ ]:
matched=0
nonmatched=0
notread=0
for fname in os.listdir(img_filepath):
    
    try:      
        image = cv2.imread(img_filepath + fname)
        fn = fname.split('.')   
        fn= fn[0][0:11]
        #print(fn)
        with graph_detection.as_default():
             with session_detection.as_default():
                predicted_text = p.predict_single_plate(session_detection,loaded_model,image,128,64)#input directory of pic   
        if ''.join(predicted_text) == fn:
            matched +=1
            #print('matched predicted=',''.join(predicted_text),', real= ',fn,", fullname=",fname)
        else:
            print('notmatched predicted=',''.join(predicted_text),', real= ',fn,", fullname=",fname)            
            try:
                #os.remove(img_filepath + fname)
                x=1
            except:
                print("remove error")            
            nonmatched +=1
    except:
        print("image not read ",img_filepath + fname)
        notread +=1
    
    
    
print('matched:', matched,",nonmatched:",nonmatched, ",not read:",notread,",accuracy:",matched/cnt_test*100,",total:",cnt_test)
    

In [ ]:
print('matched:', matched,",nonmatched:",nonmatched, ",not read:",notread,",accuracy:",matched/cnt_test*100,cnt_test)

In [ ]:
#LR=0.01 3GRU matched: 433 ,nonmatched: 238 ,not read: 0 ,accuracy: 64.53055141579732
#lr=0.02 2gru model_out5 with 2000 aug+ 1600 real -> matched: 280 ,nonmatched: 84 ,not read: 0 ,accuracy: 76.92307692307693
#4000 aug +1600 real                              -> matched: 441 ,nonmatched: 123 ,not read: 0 ,accuracy: 78.19148936170212
#with spaces matched: 438 ,nonmatched: 126 ,not read: 0 ,accuracy: 77.6595744680851

#out4 3000 augmente matched: 296 ,nonmatched: 5 ,not read: 0 ,accuracy: 98.33887043189368 301
